## Import library


In [16]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets
from tensorflow.keras import layers, models
from numpy import dot
from numpy.linalg import norm
from tensorflow import keras
import tensorflow as tf
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model

## train data

In [17]:
def sampling_func(data, max_size):
    N = len(data)
    sample_n = len(data) # integer
    if sample_n > max_size:
        sample = data.take(np.random.permutation(N)[:max_size])
    else:
        sample = data.take(np.random.permutation(N)[:len(data)])
    return sample

ID_List = pd.read_table('/Users/heogeonhyeock/Downloads/inha_data/inha_data/ID_List.txt',
                       sep = " ",
                       names = ["name", "path"])

ID_List_sample = ID_List.groupby('name', group_keys = False
                                ).apply(sampling_func, max_size = 10).reset_index()


In [18]:
train_img = []
train_name = []

for index in range(10006):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/" + ID_List_sample['path'][index]
    file = np.reshape(cv2.resize(cv2.imread(A,0), (112,112)),(112,112,1))
    train_img.append(file) # 리스트에 이미지 추가
    train_name.append(ID_List_sample['name'][index])

train_img = np.divide(np.array(train_img),255.0)
train_name = np.array(train_name)

train = dict(data = train_img,
            target = train_name)

In [19]:
# 훈련데이터 전처리
Xtrain, Ytrain, Xtarget, Ytarget = train_test_split(train['data'],
                                                train['target'], 
                                                test_size = 0.3,  stratify=train['target'])

train_name_onehot = to_categorical(Xtarget)
test_name_onehot = to_categorical(Ytarget) 

## Model

In [13]:
model = ResNet50(input_shape=(112,112,1), include_top=False, weights='imagenet', pooling='max')

x = model.output
model.trainable = False 

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(512, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(1017, activation='softmax', name='softmax')(x)

model = Model(model.input, x)
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

ValueError: The input must have 3 channels; got `input_shape=(112, 112, 1)`

In [20]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(28, kernel_size=5,
                              activation='relu',padding='samee',input_shape(112,112,1)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(56, kernel_size=5,
                              activation='relu',padding='samee',input_shape(112,112,1)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())
model.add(keras.layers.dropout(0.4))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()



SyntaxError: positional argument follows keyword argument (1016401415.py, line 4)

In [12]:
# 모델 훈련

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(Xtrain, train_name_onehot, epochs=1,
         use_multiprocessing = True)
test_loss, test_acc = model.evaluate(Ytrain, test_name_onehot, verbose=1)
print(test_acc)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Constant constructor takes either 0 or 2 positional arguments
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Constant constructor takes either 0 or 2 positional arguments
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-08-12 22:02:49.667695: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-12 22:02:49.667839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


ValueError: Input 0 of layer conv1_conv is incompatible with the layer: expected axis -1 of input shape to have value 3 but received input with shape (None, 118, 118, 1)

In [ ]:
# 모델 저장
model.save("Inha_CNN.8.11")

## Test data

In [ ]:
# 테스트 데이터 불러오기
left_test_img = []
right_test_img = []

for index in range(6000):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/left_face_" + str(index) + ".jpg"
    B = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/right_face_" + str(index) + ".jpg" 
    
    file_left = np.reshape(cv2.resize(cv2.imread(A,0), (112,112)),(112,112,1))
    file_right = np.reshape(cv2.resize(cv2.imread(B,0), (112,112)),(112,112,1))
    
    left_test_img.append(file_left)
    right_test_img.append(file_right)

left_test_img = np.divide(np.array(left_test_img),255.0)
right_test_img = np.divide(np.array(right_test_img),255.0)

## 모델평가

In [ ]:
predict_left = model.predict(left_test_img)
predict_right = model.predict(right_test_img)

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

predict = []
for i in range(len(predict_left)):
    predict_cos = cos_sim(predict_left[i],predict_right[i])
    predict.append(predict_cos)
    
submission = pd.read_csv("/Users/heogeonhyeock/Downloads/inha_data/inha_data/sample_submission.csv")
submission["answer"] = predict[:len(predict_left)]

path = '/Users/heogeonhyeock/Downloads/inha_data/inha_data'
os.chdir(path)
submission.to_csv("submission.csv", index = False)

submission